
we will take our projects from Day 1 
Business Challenge:
Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.
We will be provided a company name and their primary website.

In [1]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown,display,update_display
from openai import OpenAI

In [11]:
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key[:8] == 'sk-proj-':
    print("API Key looks good")
else:
    print("Issue getting API Key")

MODEL = 'gpt-4o-mini'
openai = OpenAI()

API Key looks good


In [5]:
# Class represent webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """Class to represent website that we have scraped , now with links"""

    def __init__(self,url):
        self.url = url
        response = requests.get(url,headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body,'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script","style","img","input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n",strip=True)
        else:
            self.text = ""
        
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]
    
    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"
        
        

In [4]:
ws = Website("https://huggingface.co/")
ws.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/google/gemma-3n-E4B-it-litert-preview',
 '/mistralai/Devstral-Small-2505',
 '/ByteDance-Seed/BAGEL-7B-MoT',
 '/multimodalart/isometric-skeumorphic-3d-bnb',
 '/nvidia/parakeet-tdt-0.6b-v2',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/Lightricks/ltx-video-distilled',
 '/spaces/NihalGazi/FLUX-Pro-Unlimited',
 '/spaces/ByteDance/DreamO',
 '/spaces/smolagents/computer-agent',
 '/spaces',
 '/datasets/openbmb/Ultra-FineWeb',
 '/datasets/disco-eth/EuroSpeech',
 '/datasets/nvidia/OpenCodeReasoning',
 '/datasets/PrimeIntellect/INTELLECT-2-RL-Dataset',
 '/datasets/nvidia/OpenMathReasoning',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/gr

Go with GPT-4o-mini figure out which links are relevant
Use a call to gpt-4o-mini to read the links on a webpage, and response in structured JSON.
It decide which links are relevant, and replace relative links such "/about" with "https://company.com/about".

This is good use case for an LLM. Without this you need to implement very complax logic to get parsed links , It would be very hard.


In [7]:
link_system_prompt = "You are provided with a list of links which is found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [8]:
print(link_system_prompt)

You are provided with a list of links which is found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [9]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
    Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [10]:
print(get_links_user_prompt(ws))

Here is the list of links on the website of https://huggingface.co/ - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format.     Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
/
/models
/datasets
/spaces
/docs
/enterprise
/pricing
/login
/join
/spaces
/models
/google/gemma-3n-E4B-it-litert-preview
/mistralai/Devstral-Small-2505
/ByteDance-Seed/BAGEL-7B-MoT
/multimodalart/isometric-skeumorphic-3d-bnb
/nvidia/parakeet-tdt-0.6b-v2
/models
/spaces/enzostvs/deepsite
/spaces/Lightricks/ltx-video-distilled
/spaces/NihalGazi/FLUX-Pro-Unlimited
/spaces/ByteDance/DreamO
/spaces/smolagents/computer-agent
/spaces
/datasets/openbmb/Ultra-FineWeb
/datasets/disco-eth/EuroSpeech
/datasets/nvidia/OpenCodeReasoning
/datasets/PrimeIntellect/INTELLECT-2-RL-Dataset
/datasets/nvidia/OpenMathReasoning
/datasets
/join
/pricing#endpoints
/pricing#spaces
/pricing
/enterprise
/enterprise
/

In [12]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = MODEL,
        messages = [
            {"role":"system","content":link_system_prompt},
            {"role":"user","content":get_links_user_prompt(website)}
        ],
        response_format={"type":"json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [13]:
huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/ByteDance-Seed/BAGEL-7B-MoT',
 '/mistralai/Devstral-Small-2505',
 '/google/gemma-3n-E4B-it-litert-preview',
 '/google/medgemma-4b-it',
 '/nari-labs/Dia-1.6B',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/Lightricks/ltx-video-distilled',
 '/spaces/NihalGazi/FLUX-Pro-Unlimited',
 '/spaces/stepfun-ai/Step1X-3D',
 '/spaces/fancyfeast/joy-caption-beta-one',
 '/spaces',
 '/datasets/openbmb/Ultra-FineWeb',
 '/datasets/disco-eth/EuroSpeech',
 '/datasets/ministere-culture/comparia-conversations',
 '/datasets/nvidia/OpenCodeReasoning',
 '/datasets/nvidia/OpenMathReasoning',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writ

In [14]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'status page', 'url': 'https://status.huggingface.co/'}]}

Now Make the brochure ....
Assemble all the details into another prompt to GPT4-o

In [ ]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result



In [23]:
print(get_all_details("https://huggingface.co"))

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}]}
get_all_details : ==> Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and applications.
Explore AI Apps
or
Browse 1M+ models
Trending on
this week
Models
ByteDance-Seed/BAGEL-7B-MoT
Updated
3 days ago
•
1.5k
•
534
mistralai/Devstral-Small-2505
Updated
2 days ago
•
64.4k
•
518
google/gemma-3n-E4B-it-litert-preview
Update

In [63]:

system_prompt = "You are an assistant tasked with analyzing key pages from a company's website and and creates a short concise brochure for potential customers,\
investors, and job seekers. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

In [64]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [65]:
def create_brochure(company_name,url):
    response = openai.chat.completions.create(
        model = MODEL,
        messages = [
            {"role":"system","content" : system_prompt},
            {"role":"user","content" : get_brochure_user_prompt(company_name,url)}
        ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [66]:
create_brochure("HuggingFace","https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}
get_all_details : ==> Landing page:
Webpage Title:
Hugging Face – The AI community building the future.
Webpage Contents:
Hugging Face
Models
Datasets
Spaces
Community
Docs
Enterprise
Pricing
Log In
Sign Up
The AI community building the future.
The platform where the machine learning community collaborates on models, datasets, and application

# Hugging Face Brochure

---

## About Us

**Hugging Face** is at the forefront of artificial intelligence, building a collaborative platform that serves the machine learning community. We provide tools, resources, and a vibrant community for researchers, developers, and businesses to advance AI technology and its applications.

---

## Our Offerings

- **Models**: Access over 1 million machine learning models, including the latest innovations from top tech companies like Google, Meta, and Microsoft.
  
- **Datasets**: Browse through 250,000+ datasets tailored for diverse machine learning tasks and research. 

- **Spaces**: Discover and create applications using the Hugging Face environment, featuring over 400,000 applications.

- **Compute Solutions**: Harness powerful GPU options starting from $0.60/hour, with enterprise-grade security and support starting at $20/user/month.

---

## The Community

Join over 50,000 organizations leveraging Hugging Face, including:

- **Non-Profits** like AI2
- Industry leaders such as Amazon, Google, and Microsoft

The Hugging Face community is built on transparency, inclusivity, and collaboration. It’s a space where anyone interested in AI can contribute, learn, and grow.

---

## Company Culture

At Hugging Face, we foster a community-oriented environment that encourages innovation and creativity. We are committed to:

- **Inclusivity**: Welcoming individuals from all backgrounds and experiences.
- **Collaboration**: Promoting teamwork and shared success through open-source tools and platforms.
- **Continuous Learning**: Supporting personal and professional development in the fast-evolving AI landscape.

---

## Careers

Be a part of shaping the future of AI! We are constantly on the lookout for talented individuals to join our diverse team. Positions in engineering, research, community management, and business development are available. 

### Current Opportunities

- Machine Learning Engineer
- Research Scientist
- Community Manager
- Software Developer

Visit our [Jobs Page](https://huggingface.co/jobs) to discover current vacancies and apply!

---

## Connect with Us

Explore our resources and become part of the Hugging Face family:

- **Website**: [huggingface.co](https://huggingface.co)
- **GitHub**: [Hugging Face GitHub](https://github.com/huggingface)
- **Social Media**: 
  - [Twitter](https://twitter.com/huggingface)
  - [LinkedIn](https://www.linkedin.com/company/huggingface)
  - [Discord Community](https://discord.gg/huggingface)

---

Together, let’s build the future of AI!